In [2]:
import numpy as np
import pandas as pd
import openml
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder

## Datasets

Small datasets:
- 37: diabetes
- 1462: banknote-authentication
- 871: pollen

Large datasets:
- 1479: hill-valey
- 1120: MagicTelescope
- 23512: higgs
- 23517: numerai28.6
- 979: waveform-5000
- 1487: ozone-level-8hr

In [87]:
def read_data(id):
    dataset = openml.datasets.get_dataset(id)
    df, _, _, _ = dataset.get_data(dataset_format="dataframe")
    numerical_cols = df.select_dtypes(include='number').columns
    target_col = df.select_dtypes(exclude='number').columns
    X = df[numerical_cols].to_numpy()
    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(df[target_col])
    return X, y

def list_data(id_list):
    data_list = []
    for id in id_list:
        data_list.append(read_data(id))
    return data_list

In [88]:
id_list = [37, 1462, 871, 1479, 1120, 23512, 23517, 979, 1487] #small datasets first
data_list = list_data(id_list)